In [13]:
from langchain_community.llms import Ollama
from langchain import PromptTemplate

model = Ollama(model='gemma2:2b', stop=["<|eot_id|>"]) # Added the stop token

In [14]:
text = input()
user_prompt = f"{text}"

system_prompt = """
Think and write your step-by-step reasoning before responding.  
Please write only the fully spelled-out form of the acronym in English that corresponds to the following user's question, without abbreviations or additional text.  

For example, if the user asks 'RAG,' output 'Retrieval-Augmented Generation,' or for 'CNN,' output 'Convolutional Neural Network.'

If the user asks about 'MoE,' the output should be 'Mixture of Experts' and nothing else.
"""

In [15]:
template = """
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    {system_prompt}
    <|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    {user_prompt}
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
"""

prompt = PromptTemplate(
    input_variables=['system_prompt', 'user_prompt'],
    template=template
)

In [16]:
key_word = model(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))

In [17]:
key_word.strip()

"Dijkstra's Algorithm"

In [18]:
f"https://en.wikipedia.org/wiki/{key_word.strip()}"

"https://en.wikipedia.org/wiki/Dijkstra's Algorithm"

In [19]:
from langchain.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = WebBaseLoader(f"https://en.wikipedia.org/wiki/{key_word.strip()}")
data = loader.load()

text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0) 
all_splits = text_splitter.split_documents(data)

In [20]:
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings

embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)
# docs = vectorstore.similarity_search("")

In [21]:
user_prompt = f"{user_prompt} on detail."

system_prompt = """
Please write all conversations in Korean(한국어).
Think and write your step-by-step reasoning before responding.
Write the article title using ## in Markdown syntax.
Please be as specific as possible in your answer.
"""

In [22]:
from langchain.chains import RetrievalQA

qachain = RetrievalQA.from_chain_type(model, retriever=vectorstore.as_retriever())

response = qachain(prompt.format(system_prompt=system_prompt, user_prompt=user_prompt))

In [23]:
response

{'query': '\n    <|begin_of_text|>\n    <|start_header_id|>system<|end_header_id|>\n    \nPlease write all conversations in Korean(한국어).\nThink and write your step-by-step reasoning before responding.\nWrite the article title using ## in Markdown syntax.\nPlease be as specific as possible in your answer.\n\n    <|eot_id|>\n    <|start_header_id|>user<|end_header_id|>\n    다익스트라 Cpp로 구현해줘 on detail.\n    <|eot_id|>\n    <|start_header_id|>assistant<|end_header_id|>\n',
 'result': '## 다익스트라 알고리즘 (C++) 구현 및 상세 설명\n\n다익스트라 알고리즘은 그래프의 각 노드 사이에 직선 거리가 가장 짧은 경로를 찾는 방법을 의미합니다. 이 알고리즘은 크기가 작거나 중간 단계에서 많은 양의 데이터 처리를 할 때 적용됩니다.  C++를 사용하여 다익스트라 알고리즘을 구현하는 방법과 상세한 설명을 보여드릴게요.\n\n**1. 다익스트라 알고리즘의 기초**\n\n다익스트라 알고리즘은 그래프의 시작 노드에서 최종 노드까지 가는 가장 빠른 경로를 찾아냅니다. 이러한 목표를 달성하기 위해,  알고리즘은 다음과 같은 단계를 거치게 됩니다:\n\n* **가장 작은 거리:** 시작 노드부터 최종 노드까지 가는 거리가 가장 작은 노드와 그 연결점을 찾기\n* **탐색 방법:** 시작 노드에서 각 노드까지의 경로를 계산하고, 최소 거리를 기준으로 노드들을 순차적으로 방문\n\n**2. C++ 구현**\n\n다익스트라 알고리즘을 구현하는 C++ 코드 예시:\n\n\n```c++\n#include <ios

In [24]:
print(response['result'])

## 다익스트라 알고리즘 (C++) 구현 및 상세 설명

다익스트라 알고리즘은 그래프의 각 노드 사이에 직선 거리가 가장 짧은 경로를 찾는 방법을 의미합니다. 이 알고리즘은 크기가 작거나 중간 단계에서 많은 양의 데이터 처리를 할 때 적용됩니다.  C++를 사용하여 다익스트라 알고리즘을 구현하는 방법과 상세한 설명을 보여드릴게요.

**1. 다익스트라 알고리즘의 기초**

다익스트라 알고리즘은 그래프의 시작 노드에서 최종 노드까지 가는 가장 빠른 경로를 찾아냅니다. 이러한 목표를 달성하기 위해,  알고리즘은 다음과 같은 단계를 거치게 됩니다:

* **가장 작은 거리:** 시작 노드부터 최종 노드까지 가는 거리가 가장 작은 노드와 그 연결점을 찾기
* **탐색 방법:** 시작 노드에서 각 노드까지의 경로를 계산하고, 최소 거리를 기준으로 노드들을 순차적으로 방문

**2. C++ 구현**

다익스트라 알고리즘을 구현하는 C++ 코드 예시:


```c++
#include <iostream>
#include <vector>
#include <queue>
#include <limits.h> 

using namespace std;

// 노드 정보
struct Node {
    int node_id; // 노드의 ID
    int distance;  // 노드에서 최소 거리
};

// 그래프를 나타내는 구조
vector<vector<pair<int, int>>> graph; 

// 다익스트라 알고리즘을 사용한 경로 계산 함수
void dijkstra(Node start) {
    // 모든 노드의 최소 거리 초기화
    vector<int> distance(graph.size(), INT_MAX); 
    distance[start.node_id] = 0;  

    // 우선순위를 위한 큐 생성
    priority_queue<pair<int, int>, vector<pair<int, int>>, greater<pair<int, int>>> pq